In [2]:
import rebound
import numpy as np
%matplotlib inline

# Make arbitrary (chaotic) sim 

In [19]:
def makesim():
    sim = rebound.Simulation()
    sim.G = 4*np.pi**2
    sim.add(m=1.)
    sim.add(m=1.e-3, P=1., e=0.01, inc=0.1, pomega=2., Omega=1., f=1.5, r=1.e-5)
    sim.add(m=1.e-3, P=1.45, e=0.01, inc=0.1, pomega=2., Omega=1., f=1.5, r=1.e-5)
    sim.move_to_com()
    sim.integrator="whfast"
    sim.dt = sim.particles[1].P*np.sqrt(3)/100.
    return sim

# Here's how we can save the simulation state with REBOUND, and reload it in a new simulation

In [20]:
sim = makesim()
sim.simulationarchive_snapshot("rebtest.bin")

In [21]:
sa = rebound.SimulationArchive("rebtest.bin")
sim2 = sa[0] # reloaded simulation

# This function integrates for 1e4 orbits, then prints the DIFFERENCE between corresponding particles in the original and reloaded simulation

In [23]:
def printdiff(sim, sim2):
    sim.integrate(1.e4)
    sim2.integrate(1.e4)
    for i in range(sim.N):
        print(sim.particles[i]-sim2.particles[i])

# We get back all 0s, meaning the trajectories are identical

In [24]:
printdiff(sim, sim2)

<rebound.Particle object, m=0.0 x=0.0 y=0.0 z=0.0 vx=0.0 vy=0.0 vz=0.0>
<rebound.Particle object, m=0.0 x=0.0 y=0.0 z=0.0 vx=0.0 vy=0.0 vz=0.0>
<rebound.Particle object, m=0.0 x=0.0 y=0.0 z=0.0 vx=0.0 vy=0.0 vz=0.0>


# Now let's try saving values with numpy. The numbers we have to set are the grav const G, the timestep, and all the particle parameters. You could make this use hdf5

In [25]:
def newsavesim(sim, filename):
    params = [sim.G, sim.dt]
    for p in sim.particles:
        params = params + [p.m, p.x, p.y, p.z, p.vx, p.vy, p.vz, p.r]
    params = np.array(params)

    params.tofile(filename)

# Here's a function that takes the above convention for saving parameters, and puts them back into a REBOUND simulation to reload it

In [26]:
def newloadsim(filename):
    loadparam = np.fromfile(filename)

    sim = rebound.Simulation()
    sim.G = loadparam[0]
    sim.dt = loadparam[1]

    Nglobals = 2 # G and dt above
    Nbodies = 3
    Nparams = 8 # mass, radius and 6 cartesian pos/vel
    for j in range(Nbodies):
        p = loadparam[Nglobals + j*Nparams:Nglobals + (j+1)*Nparams] # get parameters for next planet
        sim.add(m=p[0], x=p[1], y=p[2], z=p[3], vx=p[4], vy=p[5], vz=p[6], r=p[7])
    sim.integrator="whfast"
    return sim

In [27]:
sim = makesim()
newsavesim(sim, 'test.bin')
sim2 = newloadsim('test.bin')

# Reloads and gives back identical trajectory fine

In [30]:
printdiff(sim, sim2)

<rebound.Particle object, m=0.0 x=0.0 y=0.0 z=0.0 vx=0.0 vy=0.0 vz=0.0>
<rebound.Particle object, m=0.0 x=0.0 y=0.0 z=0.0 vx=0.0 vy=0.0 vz=0.0>
<rebound.Particle object, m=0.0 x=0.0 y=0.0 z=0.0 vx=0.0 vy=0.0 vz=0.0>
